In [70]:
!curl ipinfo.io

{
  "ip": "34.32.223.199",
  "hostname": "199.223.32.34.bc.googleusercontent.com",
  "city": "Groningen",
  "region": "Groningen",
  "country": "NL",
  "loc": "53.2192,6.5667",
  "org": "AS396982 Google LLC",
  "postal": "9711",
  "timezone": "Europe/Amsterdam",
  "readme": "https://ipinfo.io/missingauth"
}

In [71]:
!pip install sentence-transformers torch
!pip install datasets

In [72]:
#!pip install pinecone-client==2.0.10  #pinecone-client<=2.0.10

In [73]:
!pip install pinecone-client==2.2.4

In [74]:
import pinecone
print("Pinecone version:", pinecone.__version__)

Pinecone version: 2.2.4


In [75]:
!pip install sacremoses #TransfoXLTokenizer requires the sacremoses library

In [76]:
import datasets as ds
quora = ds.load_dataset("quora", split="train", download_mode="force_redownload")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/404290 [00:00<?, ? examples/s]

In [77]:
len(quora)

404290

In [78]:
quora[5] #same example

{'questions': {'id': [11, 12],
  'text': ['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?',
   "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"]},
 'is_duplicate': True}

In [79]:
quora[30] #different example

{'questions': {'id': [61, 62],
  'text': ["What's one thing you would like to do better?",
   "What's one thing you do despite knowing better?"]},
 'is_duplicate': False}

In [80]:
import pandas as pd

df = quora.to_pandas()

In [81]:
df.head()

,questions,is_duplicate
0,"{'id': [1, 2], 'text': ['What is the step by s...",False
1,"{'id': [3, 4], 'text': ['What is the story of ...",False
2,"{'id': [5, 6], 'text': ['How can I increase th...",False
3,"{'id': [7, 8], 'text': ['Why am I mentally ver...",False
4,"{'id': [9, 10], 'text': ['Which one dissolve i...",False


In [82]:
#extracting the text
df.columns

Index(['questions', 'is_duplicate'], dtype='object')

In [83]:
df['questions'][0]

{'id': array([1, 2], dtype=int32),
 'text': array(['What is the step by step guide to invest in share market in india?',
        'What is the step by step guide to invest in share market?'],
       dtype=object)}

In [84]:
import numpy as np

def extract_text_from_numpy_dict(data):
    # Check if the input is a dictionary-like structure
    if isinstance(data, dict):
        # Check if the 'text' key exists and if it contains a NumPy array
        if 'text' in data and isinstance(data['text'], np.ndarray):
            return data['text'].tolist()  # Convert the NumPy array to a Python list
    return None  # Return None if the structure doesn't match

In [85]:
text_part = extract_text_from_numpy_dict(df['questions'][1])

In [86]:
print(text_part)

['What is the story of Kohinoor (Koh-i-Noor) Diamond?', 'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?']


In [87]:
#for iteration through a column

def extract_text_from_numpy_dict_column(dataframe, column_name):
    text_parts = []

    for index, row in dataframe.iterrows():
        data = row[column_name]

        # Check if the input is a dictionary-like structure
        if isinstance(data, dict):
            # Check if the 'text' key exists and if it contains a NumPy array
            if 'text' in data and isinstance(data['text'], np.ndarray):
                text_parts.append(data['text'].tolist())  # Convert the NumPy array to a Python list
            else:
                text_parts.append(None)
        else:
            text_parts.append(None)

    return text_parts


In [88]:
text_parts = extract_text_from_numpy_dict_column(df, 'questions')

# Display the result
'''
for i, text_part in enumerate(text_parts):
    print(f"Row {i + 1}: {text_part}")
'''

'\nfor i, text_part in enumerate(text_parts):\n    print(f"Row {i + 1}: {text_part}")\n'

In [89]:
len(text_parts)

404290

In [90]:
text_parts[0:5]

[['What is the step by step guide to invest in share market in india?',
  'What is the step by step guide to invest in share market?'],
 ['What is the story of Kohinoor (Koh-i-Noor) Diamond?',
  'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?'],
 ['How can I increase the speed of my internet connection while using a VPN?',
  'How can Internet speed be increased by hacking through DNS?'],
 ['Why am I mentally very lonely? How can I solve it?',
  'Find the remainder when [math]23^{24}[/math] is divided by 24,23?'],
 ['Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?',
  'Which fish would survive in salt water?']]

In [91]:
text_parts[0]

['What is the step by step guide to invest in share market in india?',
 'What is the step by step guide to invest in share market?']

In [92]:
text2 = [text.split(',') for text in text_parts[0]]
text2[0]

['What is the step by step guide to invest in share market in india?']

In [93]:
'''
def split_text_in_dataframe_column(dataframe, column_name):
    text_parts = extract_text_from_numpy_dict_column(dataframe, column_name)
    text2_column = []

    for text_part in text_parts:
        if text_part is not None:
            text2 = [text.split(', ') for text in text_part]
        else:
            text2 = []
        text2_column.append(text2)

    dataframe['text2'] = text2_column
'''

"\ndef split_text_in_dataframe_column(dataframe, column_name):\n    text_parts = extract_text_from_numpy_dict_column(dataframe, column_name)\n    text2_column = []\n\n    for text_part in text_parts:\n        if text_part is not None:\n            text2 = [text.split(', ') for text in text_part]\n        else:\n            text2 = []\n        text2_column.append(text2)\n\n    dataframe['text2'] = text2_column\n"

In [94]:
import pandas as pd
df = pd.DataFrame(text_parts)
df.head()

,0,1
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [95]:
import pandas as pd

def convert_to_interleaved_data(df):
    if len(df.columns) != 2:
        raise ValueError("The DataFrame must have exactly two columns.")

    column1_values = df[df.columns[0]].tolist()
    column2_values = df[df.columns[1]].tolist()

    interleaved_data = [val for pair in zip(column1_values, column2_values) for val in pair]

    interleaved_dict = {df.columns[0]: interleaved_data}

    return pd.DataFrame(interleaved_dict)

# Example usage:
'''
data = {'Column1': ['Hello', 'World', 'This', 'Is'], 'Column2': ['123', '456', '789', '000']}
df_test = pd.DataFrame(data)
result_df = convert_to_interleaved_data(df_test)
print(result_df)
'''

"\ndata = {'Column1': ['Hello', 'World', 'This', 'Is'], 'Column2': ['123', '456', '789', '000']}\ndf_test = pd.DataFrame(data)\nresult_df = convert_to_interleaved_data(df_test)\nprint(result_df)\n"

In [96]:
id2text = convert_to_interleaved_data(df)

In [97]:
id2text.head()

,0
0,What is the step by step guide to invest in sh...
1,What is the step by step guide to invest in sh...
2,What is the story of Kohinoor (Koh-i-Noor) Dia...
3,What would happen if the Indian government sto...
4,How can I increase the speed of my internet co...


In [98]:
id2text[0][0]

'What is the step by step guide to invest in share market in india?'

In [99]:
len(id2text)

808580

**Get model: all-distilroberta-v1**

from huggingface intended to be used as a sentence and short paragraph encoder

In [100]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-distilroberta-v1")

**Generating Embeddings and Upsert**

In [101]:
#example embeddings

text = "what's the one thing you do despite knowing better?"
embeddings = model.encode(text).tolist()
print(embeddings)  #768 values

[0.04842783883213997, 0.00647372892126441, -0.051976002752780914, -0.025520512834191322, 0.04331560060381889, -0.06516124308109283, -0.0038804528303444386, -0.04953460022807121, 0.09117073565721512, 0.013322383165359497, -0.0046014380641281605, -0.05372092127799988, 0.013246721588075161, 0.020778566598892212, -0.06645707786083221, 0.04841157793998718, 0.018589278683066368, -0.0208139568567276, 0.020627325400710106, -0.004534528590738773, 0.011303032748401165, 0.04132404550909996, -0.01914716511964798, -0.02042854018509388, 0.04341401904821396, 0.004532219842076302, 0.0050032357685267925, 0.026532599702477455, 0.035849519073963165, -0.021335193887352943, -0.007900618948042393, 0.02434510923922062, -0.02910381555557251, 0.04047751426696777, -0.007789685856550932, -0.021850235760211945, 0.0038260796573013067, -0.027652118355035782, 0.025395698845386505, -0.0030118818394839764, -0.08460874110460281, -0.026168687269091606, 0.012439027428627014, -0.05211656168103218, -0.027483252808451653, 0

In [102]:
len(embeddings)

768

In [103]:
#organize into vector
id = "vec62"
values = embeddings
metadata = {
    'is_duplicate':False,
    'char_length': len(text)
}

vector = (id, values, metadata)
print(vector)

('vec62', [0.04842783883213997, 0.00647372892126441, -0.051976002752780914, -0.025520512834191322, 0.04331560060381889, -0.06516124308109283, -0.0038804528303444386, -0.04953460022807121, 0.09117073565721512, 0.013322383165359497, -0.0046014380641281605, -0.05372092127799988, 0.013246721588075161, 0.020778566598892212, -0.06645707786083221, 0.04841157793998718, 0.018589278683066368, -0.0208139568567276, 0.020627325400710106, -0.004534528590738773, 0.011303032748401165, 0.04132404550909996, -0.01914716511964798, -0.02042854018509388, 0.04341401904821396, 0.004532219842076302, 0.0050032357685267925, 0.026532599702477455, 0.035849519073963165, -0.021335193887352943, -0.007900618948042393, 0.02434510923922062, -0.02910381555557251, 0.04047751426696777, -0.007789685856550932, -0.021850235760211945, 0.0038260796573013067, -0.027652118355035782, 0.025395698845386505, -0.0030118818394839764, -0.08460874110460281, -0.026168687269091606, 0.012439027428627014, -0.05211656168103218, -0.02748325280

In [104]:
#connecting to pinecone
with open('API_KEY.py') as f:
  API_KEY = f.readline()

import pinecone
pinecone.init(api_key=API_KEY, environment='gcp-starter') #just next to API_KEY in console
#pinecone.create_index(name= 'semantic-search', dimention=768)
index = pinecone.Index('semantic-search')

In [105]:
print(API_KEY)

df3924c8-2776-4311-b078-4b56004fe8e6


In [106]:
#upsert inex
index.upsert(vectors=[vector])

{'upserted_count': 1}

In [107]:
#check index
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.43922,
 'namespaces': {'': {'vector_count': 43922}},
 'total_vector_count': 43922}

** For UPSERT More Data**

In [108]:
type(quora)

datasets.arrow_dataset.Dataset

In [109]:
quora.shape

(404290, 2)

In [110]:
quora_subset = quora.select([i for i in range(100)])

In [111]:
type(quora_subset)

datasets.arrow_dataset.Dataset

In [112]:
quora_subset.shape

(100000, 2)

In [113]:

upsert_more = True

if upsert_more:
  from tqdm.auto import tqdm
  data = []
  #looping through and create json files
  for i, row in enumerate(tqdm(quora_subset)):
    #each quora row contains a pair of sentence,loop through both
    for pair in [0, 1]:
      text = row['questions']['text'][pair]
      #appending the (id, vector, metadata) tuple to our data list
      data.append((
          str(row['questions']['id'][pair]),
          model.encode(text).tolist(),
          {
              'is_duplicate': int(row['is_duplicate']),
              'char_length': len(text)
          }
      ))
    #once we reach the end of dataset or 100 samples, upsert to pinecone
    if len(data) == 100 or i == len(quora):
      index.upsert(vectors = data)
      #and now reset the data list
      data = []


  0%|          | 0/100000 [00:00<?, ?it/s]

ApiException: ignored

In [ ]:
index.describe_index_stats()

In [ ]:
print("Pinecone version:", pinecone.__version__)

**Query**

query vector

In [ ]:
query = "which quora queries are good??"
xq = model.encode([query]).tolist()

In [ ]:
result = index.query(xq, top_k=5)
print(result)

In [ ]:
result1 = {'results': [result]}
print(result1)

In [ ]:
result1['results'][0]['matches']

In [ ]:
# results related to query vector
for item in result1['results'][0]['matches']:
  print(item.id)

In [ ]:
id2text[0][28712]

In [ ]:
id2text[0][34002]

**Using Keyword**

In [ ]:
#example embedding
text  = "what's one thing you do despite knowing better?"

In [ ]:
!pip install sacremoses #TransfoXLTokenizer requires the sacremoses library

In [ ]:
#tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('transfo-xl-wt103')

In [ ]:
#example tokenization
tokens= tokenizer.tokenize(text.lower())
print(tokens)

In [ ]:
#organize into vectors
id = "vec63"
values = model.encode(text).tolist()

#passing tokens as metadata

metadata = {
    'is_duplicate':False,
    'char_length': len(text),
    'tokens': tokenizer.tokenize(text.lower()),
}

vextor =(id, values, metadata)
print(vector)

In [ ]:
print("Pinecone version:", pinecone.__version__)

**Applying Keyword Filtering**

In [ ]:
query = "how to ask a good questions?"
xq = model.encode([query]).tolist()

result = index.query(xq, top_k=10, filter={'tokens':{
    '$in':['reddit']
}})


In [ ]:
result2 = {'results': [result]}
print(result2)

In [ ]:
result2 = {'results': [result]}
print(result2)
#question with reddit keyword in them
for item in result2['results'][0]['matches']:
  print(item.id)

In [ ]:
query = "how to ask a good question??"

xq = model.encode([query]).tolist()

result = index.query(xq, top_k=10, filter={'$and':[
                                           {'tokens':{'$in':["reddit"]}},
                                           {'is_duplicate':{'$eq':0 }}
                                           ]})



In [ ]:
result2 = {'results': [result]}
print(result2)
#question with reddit keyword in them
for item in result2['results'][0]['matches']:
  print(item.id)

In [ ]:
qquery = "can u teach a python how to code in python?"
xq = model.encode([qquery]).tolist()

result = index.query(xq, top_k =5)
'''
for item in result['results'][0]['matches']:
  print(id2text[item.id])
'''

In [ ]:
result2 = {'results': [result]}
print(result2)
#question with reddit keyword in them
for item in result2['results'][0]['matches']:
  print(item.id)

In [ ]:
query = "can u teach a snake to program code in python?"
xq = model.encode([query]).tolist()

toks = tokenizer.tokenize(query.lower())
print(toks)

In [ ]:
#find question talk about snakes

if 'snake' in toks:
  result = index.query(xq, top_k=5, filter={'$and':[
      {'token': {'$in':['snakes']}}
  ]})

else:
  result = index.query(xq, top_k=5)
  result2 = {'results': [result]}
  print(result2)
  #question with reddit keyword in them
  for item in result2['results'][0]['matches']:
    print(item.id)


In [ ]:
print("notebook Complete")